# 14. Volatility Surface

Import libraries

In [1]:
import pyodbc
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d

Import data from SQL DataBase

In [157]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                      "Server=BDPRADA.LACAISSE.COM;"
                      "Database=ADA_BETA;"
                      "Trusted_Connection=yes;")

df = pd.read_sql_query('SELECT * FROM [sandbox_analyse_quantitative].[Swaption_v1]',cnxn)

Change column 'expiry' and 'tenor' for numeric

In [159]:
expYears = df['expiry'].str.contains('Y')
tenYears = df['tenor'].str.contains('Y')

In [161]:
rep = {'expiry': {'[YM]':''}, 'tenor': {'[YM]':''}}
df.replace(rep, inplace=True, regex=True)
df[['expiry', 'tenor']] = df[['expiry', 'tenor']].apply(pd.to_numeric)

In [162]:
df.loc[expYears,'expiry'] = df.loc[expYears,'expiry']*12
df.loc[tenYears,'tenor'] = df.loc[tenYears,'tenor']*12

Set datatype to relevant information

In [163]:
df.set_index(['RIC'], inplace = True)
df.vDate = pd.to_datetime(df.vDate)
df.relative_strike = pd.to_numeric(df.relative_strike)
df.dtypes

ccy                        object
expiry                      int64
tenor                       int64
relative_strike             int64
vDate              datetime64[ns]
Norm_Vol_JPM              float64
Norm_Vol_TP               float64
Norm_vol_Citi             float64
dtype: object

In [164]:
df.head()

,ccy,expiry,tenor,relative_strike,vDate,Norm_Vol_JPM,Norm_Vol_TP,Norm_vol_Citi
RIC,,,,,,,,
SSSR1USD02Y30Y,USD,24,360,-100,2002-01-18,105.132832,NaN,NaN
SOSWOEUR06M02Y,EUR,6,24,0,2015-08-28,NaN,19.64,19.130048
SSSPHEUR01Y05Y,EUR,12,60,50,2010-04-19,NaN,71.70,78.237724
SSSPHEUR01Y02Y,EUR,12,24,50,2016-05-06,NaN,37.72,37.093097
SSSRQUSD01Y05Y,USD,12,60,-25,2005-08-18,94.887327,NaN,94.376787


In [165]:
USD01M = df.loc[ (df.ccy == 'USD') & (df.expiry == 1) & (df.vDate == '2016-05-06') , : ].copy()
USD01M.sort_values (['tenor','relative_strike'], inplace = True)
USD01M

,ccy,expiry,tenor,relative_strike,vDate,Norm_Vol_JPM,Norm_Vol_TP,Norm_vol_Citi
RIC,,,,,,,,
SSSR1USD01M02Y,USD,1,24,-100,2016-05-06,61.029545,0.00,51.467575
SSSRHUSD01M02Y,USD,1,24,-50,2016-05-06,44.634036,44.63,46.079877
SSSRQUSD01M02Y,USD,1,24,-25,2016-05-06,47.107285,45.34,43.636590
SOSWOUSD01M02Y,USD,1,24,0,2016-05-06,49.413216,47.68,45.335363
SSSPQUSD01M02Y,USD,1,24,25,2016-05-06,52.073148,51.73,51.007573
SSSPHUSD01M02Y,USD,1,24,50,2016-05-06,53.422323,56.86,58.314531
SSSP1USD01M02Y,USD,1,24,100,2016-05-06,NaN,68.23,73.567377
SSSR1USD01M05Y,USD,1,60,-100,2016-05-06,79.043937,67.19,92.212907
SSSRHUSD01M05Y,USD,1,60,-50,2016-05-06,72.818273,66.00,76.332390


In [205]:
fig = plt.figure()
ax1 = fig.add_subplot(111, projection = '3d')

x = USD01M.tenor
y = USD01M.relative_strike
z = np.zeros(len(USD01M))

dx = np.ones(len(USD01M))
dy = np.ones(len(USD01M))
dz = USD01M.Norm_Vol_JPM


ax1.bar3d (x,y,z,dx,dy,dz)

plt.show()


In [206]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
import numpy as np
import scipy as si

from scipy import interpolate
x = np.arange(-5.01, 5.01, 0.25)
y = np.arange(-5.01, 5.01, 0.25)
xx, yy = np.meshgrid(x, y)
z = np.sin(xx**2+yy**2)
f = interpolate.interp2d(x, y, z, kind='cubic')

xnew = np.arange(-5.01, 5.01, 1e-2)
ynew = np.arange(-5.01, 5.01, 1e-2)
znew = f(xnew, ynew)
plt.plot(x, z[0, :], 'ro-', xnew, znew[0, :], 'b-')
plt.show()

In [207]:
import pylab
import scipy as si
from mpl_toolkits.mplot3d import Axes3D

x = si.linspace (-3,3,300)
y = si.linspace (-3,3,300)
[x,y] =si.meshgrid(x,y)
z = si.sqrt(x**2+y**2)

fig = pylab.figure()

ax = Axes3D(fig)
ax.plot_surface(x,y,z)
